In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
import itertools
from pyspark.sql.functions import col, when, least, greatest, lit

In [1]:
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from sparknlp.annotator import StopWordsCleaner
from pyspark.sql import types as T
from pyspark.sql import functions as F
from sparknlp.annotator import NGramGenerator
from sparknlp.base import Finisher
from pyspark.ml.tuning import ParamGridBuilder

spark = sparknlp.start()

data = spark.read.csv("../data/cleaned_moral_scores.csv", header= True).select(["id", "cleaned_text"])

#Preprocessing
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') 

english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "can", "could", "may",
        "might", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", 
        "shant", "shed", "shell", "shes", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "even", "one", "bye", "take", "wasnt"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year", "now"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement", "blah"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot', "selfimprovement", "_", "ampxb"]

stopwords = english + time + reddit + topic_specific

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['normalized']) \
     .setOutputCol('unigrams') \
     .setStopWords(stopwords)

pos = PerceptronModel.load("/project/macs40123/spark-jars/pos_anc_en_3.0.0_3.0_1614962126490/")\
      .setInputCols("document", "unigrams")\
      .setOutputCol("pos")

finisher = Finisher().setInputCols(['unigrams', 'pos'])

my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          stopwords_cleaner,
          #ngrammer,
          pos,
          finisher
      ])

pipelineModel = my_pipeline.fit(data)
processed_data = pipelineModel.transform(data)
processed_data.persist()

#Filter by POS
def filter_unigrams(finished_unigrams, finished_pos):
    '''Filters individual words based on their POS tag'''
    return [word for word, pos in zip(finished_unigrams, finished_pos)
            if pos in ['JJ', 'NN', 'NNS', 'NNPS']]

udf_filter_unigrams = F.udf(filter_unigrams, T.ArrayType(T.StringType()))

processed_data = processed_data.withColumn('filtered_unigrams_by_pos', udf_filter_unigrams(
                                                   F.col('finished_unigrams'),
                                                   F.col('finished_pos')))

#Now that POS was done, lemmatization makes more sense at this point

#Merge tokens as just one string to be able to take it as a document in the new Pipeline
tokens_as_string = F.udf(lambda x: ' '.join(x), T.StringType())
processed_data = processed_data.withColumn('joined_tokens', tokens_as_string(F.col('filtered_unigrams_by_pos')))

last_documentAssembler = DocumentAssembler() \
     .setInputCol('joined_tokens') \
     .setOutputCol('joined_document')

last_tokenizer = Tokenizer() \
     .setInputCols(['joined_document']) \
     .setOutputCol('tokenized')
     
lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("tokenized")\
      .setOutputCol("lemmatized")

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('final') \
     .setStopWords(stopwords)

last_finisher = Finisher() \
     .setInputCols(['final']) \

last_pipeline = Pipeline() \
     .setStages([last_documentAssembler,                  
                 last_tokenizer,
                 lemmatizer,
                 stopwords_cleaner,
                 last_finisher])

final_data = last_pipeline.fit(processed_data).transform(processed_data)

processed_data.unpersist()
final_data.persist()

## Vectorization
#Apply TF-IDF filtering
tfizer = CountVectorizer(inputCol='finished_final', outputCol='tf_features', minDF=0.01, maxDF=0.80, vocabSize= 2000)
tf_model = tfizer.fit(final_data)
tf_result = tf_model.transform(final_data)

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

final_data.unpersist()
tfidf_result.persist()

24/12/03 18:25:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


DataFrame[id: string, cleaned_text: string, finished_unigrams: array<string>, finished_pos: array<string>, filtered_unigrams_by_pos: array<string>, joined_tokens: string, finished_final: array<string>, tf_features: vector, tf_idf_features: vector]

Asked ChatGPT "how can I filter tokenized words from TF IDF?"

In [6]:
vocabulary = tf_model.vocabulary

# Define a function to filter words by their TF-IDF score
# UDF to map indices to words using the vocabulary
def filter_tfidf(features, threshold=1, vocabulary=None):
    if features is not None:
        # Filter based on TF-IDF score and map indices to actual words
        return [vocabulary[features.indices[i]] for i in range(len(features.values)) if features.values[i] >= threshold]
    return []

# Register the UDF
filter_udf = udf(lambda features: filter_tfidf(features, threshold=1, vocabulary=vocabulary), ArrayType(StringType()))

# Apply the filtering function
df_filtered_tfidf = tfidf_result.withColumn("filtered_words_tfidf", filter_udf("tf_idf_features"))

In [7]:
df_filtered_tfidf.show()

+-----+--------------------+--------------------+--------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   id|        cleaned_text|   finished_unigrams|        finished_pos|filtered_unigrams_by_pos|       joined_tokens|      finished_final|         tf_features|     tf_idf_features|filtered_words_tfidf|
+-----+--------------------+--------------------+--------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|hk5r2|i had an appointm...|[appointment, den...|[NN, NN, JJ, NN, ...|    [appointment, den...|appointment denti...|[appointment, den...|(533,[17,35,99,18...|(533,[17,35,99,18...|[sure, last, phon...|
|iqimz|i created this si...|[created, site, s...|[VBN, NN, JJ, NNS...|    [site, several, m...|site several mont...|[site, several, r...|(533,[58,189,190,...|(533,[58,189,190,...|[future, helpful,

In [8]:
tfidf_result.unpersist()

DataFrame[id: string, cleaned_text: string, finished_unigrams: array<string>, finished_pos: array<string>, filtered_unigrams_by_pos: array<string>, joined_tokens: string, finished_final: array<string>, tf_features: vector, tf_idf_features: vector]

Create network - Asked chatgpt how to create all possible pair combinations between a list of words

In [9]:
import itertools
def generate_edges(tokens):
    return [list(pair) for pair in itertools.combinations(tokens, 2)]

generate_edges_udf = udf(generate_edges, ArrayType(ArrayType(StringType())))

In [10]:
df_edges = df_filtered_tfidf.withColumn("edges", generate_edges_udf(F.col("filtered_words_tfidf")))

In [11]:
df_flat_edges = df_edges.select(
    F.col("id"),
    F.explode(F.col("edges")).alias("edge")
)

In [12]:
edges_df = df_flat_edges.select(
    F.col("id").alias("id_doc"),
    F.col("edge")[0].alias("node1"),
    F.col("edge")[1].alias("node2")
)


Now, need to aggregate edges by count

In [13]:
edges_df = edges_df.withColumn("weight", lit(1))
edges_df.show(10)

+------+-----+-----+------+
|id_doc|node1|node2|weight|
+------+-----+-----+------+
| hk5r2| sure| last|     1|
| hk5r2| sure|phone|     1|
| hk5r2| sure| call|     1|
| hk5r2| sure|smoke|     1|
| hk5r2| last|phone|     1|
| hk5r2| last| call|     1|
| hk5r2| last|smoke|     1|
| hk5r2|phone| call|     1|
| hk5r2|phone|smoke|     1|
| hk5r2| call|smoke|     1|
+------+-----+-----+------+
only showing top 10 rows



In [14]:
# Normalize the pairs: ensure node1 is always less than node2, so they can be always on the same order
edges_df = edges_df.withColumn("node1_norm", least(col("node1"), col("node2"))) \
             .withColumn("node2_norm", greatest(col("node1"), col("node2")))

edges_df.show(10)

+------+-----+-----+------+----------+----------+
|id_doc|node1|node2|weight|node1_norm|node2_norm|
+------+-----+-----+------+----------+----------+
| hk5r2| sure| last|     1|      last|      sure|
| hk5r2| sure|phone|     1|     phone|      sure|
| hk5r2| sure| call|     1|      call|      sure|
| hk5r2| sure|smoke|     1|     smoke|      sure|
| hk5r2| last|phone|     1|      last|     phone|
| hk5r2| last| call|     1|      call|      last|
| hk5r2| last|smoke|     1|      last|     smoke|
| hk5r2|phone| call|     1|      call|     phone|
| hk5r2|phone|smoke|     1|     phone|     smoke|
| hk5r2| call|smoke|     1|      call|     smoke|
+------+-----+-----+------+----------+----------+
only showing top 10 rows



In [16]:
edges_df = edges_df.groupBy("node1_norm", "node2_norm").sum("weight") \
                        .withColumnRenamed("sum(weight)", "weight")

edges_df.orderBy('weight', aescending=False).show(10)

+----------+-----------+------+
|node1_norm| node2_norm|weight|
+----------+-----------+------+
|boundaries|   computer|    29|
|       bro|     manage|    30|
|       bro|       tool|    30|
|       bro|       tend|    30|
|boundaries|        fat|    33|
|       bro|friendships|    33|
|   awkward|       meal|    34|
|       bro|  unhealthy|    35|
|       bro|   computer|    35|
|       bro|   horrible|    35|
+----------+-----------+------+
only showing top 10 rows



In [17]:
edges_df.write.mode("overwrite").csv("edges_network7", header=True)

Create nodes

In [18]:
vertices_df = edges_df.select(F.col("node1_norm").alias("node")).union(edges_df.select(F.col("node2_norm").alias("node"))).distinct()
vertices_df.write.mode("overwrite").csv("nodes_network7", header=True)

In [19]:
vertices_df.show(10)

+------------+
|        node|
+------------+
|      online|
|        hope|
|       inner|
|  productive|
|conversation|
|     achieve|
|      growth|
|       watch|
| perspective|
|   character|
+------------+
only showing top 10 rows

